In [47]:
!pip install gym[atari]
!pip install autorom[accept-rom-license]
!pip install opencv-python
!pip install torch 
!pip install stable-baselines3
!pip install shimmy



[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [48]:
import gym
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import random
import cv2
from collections import deque
from tqdm import tqdm
from stable_baselines3.common.vec_env import DummyVecEnv  # For parallel environments

# Frame preprocessing function
def preprocess_frame(frame):
    frame = cv2.cvtColor(frame, cv2.COLOR_RGB2GRAY)  # Convert to grayscale
    frame = cv2.resize(frame, (84, 84))  # Resize to 84x84
    return frame / 255.0  # Normalize pixel values to [0, 1]

# Define the DQN Network
class DQN(nn.Module):
    def __init__(self, input_shape, num_actions):
        super(DQN, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(input_shape[0], 32, kernel_size=8, stride=4),
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=4, stride=2),
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, stride=1),
            nn.ReLU()
        )
        conv_out_size = self._get_conv_out(input_shape)
        self.fc = nn.Sequential(
            nn.Linear(conv_out_size, 512),
            nn.ReLU(),
            nn.Linear(512, num_actions)
        )

    def _get_conv_out(self, shape):
        o = torch.zeros(1, *shape)
        o = self.conv(o)
        return int(np.prod(o.size()))

    def forward(self, x):
        conv_out = self.conv(x).view(x.size()[0], -1)
        return self.fc(conv_out)

# Experience Replay Buffer
class ReplayBuffer:
    def __init__(self, capacity):
        self.buffer = deque(maxlen=capacity)

    def push(self, state, action, reward, next_state, done):
        self.buffer.append((state, action, reward, next_state, done))

    def sample(self, batch_size):
        batch = random.sample(self.buffer, batch_size)
        states, actions, rewards, next_states, dones = zip(*batch)
        return np.array(states), actions, rewards, np.array(next_states), dones

    def __len__(self):
        return len(self.buffer)

# Main DQN agent class
class DQNAgent:
    def __init__(self, env, buffer_size=1000000, batch_size=64, gamma=0.99, lr=1e-4, epsilon=1.0, epsilon_min=0.1, epsilon_decay=0.995):
        self.env = env
        self.memory = ReplayBuffer(buffer_size)
        self.batch_size = batch_size
        self.gamma = gamma
        self.epsilon = epsilon
        self.epsilon_min = epsilon_min
        self.epsilon_decay = epsilon_decay
        self.num_actions = env.action_space.n

        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

        # Initialize DQN and target DQN
        input_shape = (4, 84, 84)  # 4 stacked frames of 84x84
        self.policy_net = DQN(input_shape, self.num_actions).to(self.device)
        self.target_net = DQN(input_shape, self.num_actions).to(self.device)
        self.target_net.load_state_dict(self.policy_net.state_dict())
        self.target_net.eval()

        self.optimizer = optim.Adam(self.policy_net.parameters(), lr=lr)

    def select_action(self, state):
        if random.random() < self.epsilon:
            return self.env.action_space.sample()
        else:
            state = torch.FloatTensor(state).unsqueeze(0).to(self.device)
            with torch.no_grad():
                q_values = self.policy_net(state)
            return q_values.max(1)[1].item()

    def store_experience(self, state, action, reward, next_state, done):
        self.memory.push(state, action, reward, next_state, done)

    def update_target_net(self):
        self.target_net.load_state_dict(self.policy_net.state_dict())

    def train_step(self):
        if len(self.memory) < self.batch_size:
            return

        # Parallelized batch processing
        states, actions, rewards, next_states, dones = self.memory.sample(self.batch_size)

        states = torch.FloatTensor(states).to(self.device)
        actions = torch.LongTensor(actions).to(self.device)
        rewards = torch.FloatTensor(rewards).to(self.device)
        next_states = torch.FloatTensor(next_states).to(self.device)
        dones = torch.FloatTensor(dones).to(self.device)

        # Q-value prediction
        q_values = self.policy_net(states)
        next_q_values = self.target_net(next_states)

        q_value = q_values.gather(1, actions.unsqueeze(1)).squeeze(1)
        next_q_value = next_q_values.max(1)[0]

        # Bellman equation
        expected_q_value = rewards + (1 - dones) * self.gamma * next_q_value

        # Compute loss
        loss = nn.MSELoss()(q_value, expected_q_value.detach())

        # Optimize the model
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()


# Create a parallelized environment
def make_env(env_name):
    def _init():
        env = gym.make(env_name)
        return env
    return _init



In [63]:
env_name="PongNoFrameskip-v4"
num_episodes=150
target_update=10
num_envs=8
history=[]
envs = DummyVecEnv([make_env(env_name) for _ in range(num_envs)])  # Vectorized environments
agent = DQNAgent(envs)

from torch.cuda.amp import GradScaler, autocast

# Initialize a gradient scaler
scaler = GradScaler("cuda")

for episode in tqdm(range(num_episodes)):
    states = envs.reset()
    states = np.stack([preprocess_frame(s) for s in states], axis=0)
    states = np.stack([states] * 4, axis=1)  # Stack frames for input
    episode_rewards = np.zeros(num_envs)

    while True:
        actions = [agent.select_action(state) for state in states]
        next_states, rewards, dones, _ = envs.step(actions)
        next_states = np.stack([preprocess_frame(ns) for ns in next_states], axis=0)
        next_states = np.concatenate([states[:, 1:], np.expand_dims(next_states, axis=1)], axis=1)

        # Store experiences and perform mixed-precision training
        for i in range(num_envs):
            agent.store_experience(states[i], actions[i], rewards[i], next_states[i], dones[i])

        # Mixed precision training step
        with autocast():  # Enable autocast for mixed precision
            agent.train_step()  # Ensure train_step uses scaler.scale(loss).backward() internally if computing loss

        # Update states and rewards
        states = next_states
        episode_rewards += rewards
        agent.epsilon = max(agent.epsilon_min, agent.epsilon * agent.epsilon_decay)    

        if dones.any():
            break

    

    if episode % target_update == 0:
        agent.update_target_net()

        
    for i in range(num_envs): 
        print(f"Episode: {episode}, Reward: {episode_rewards[i]:.2f}, Epsilon: {agent.epsilon:.2f}")
        history.append(episode_rewards[i])

envs.close()

torch.save(agent.policy_net.state_dict(), 'dqn_policy_net.pth')  # Save the policy network
torch.save(agent.target_net.state_dict(), 'dqn_target_net.pth')  # Save the target network
torch.save(agent.optimizer.state_dict(), 'dqn_optimizer.pth')     # Save the optimizer state

# Save the history list as a .npy file
np.save('dqn_history.npy', history)  # Save the history list    

c:\Users\draka\Downloads\CODES\projects\virtual_environment\lib\site-packages\stable_baselines3\common\vec_env\patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(
C:\Users\draka\AppData\Local\Temp\ipykernel_18220\2944413868.py:12: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler("cuda")
  0%|          | 0/150 [00:00<?, ?it/s]c:\Users\draka\Downloads\CODES\projects\virtual_environment\lib\site-packages\gym\utils\passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):
C:\Users\draka\AppData\Local\Temp\ipykernel_18220\2944413868.py:31: FutureWarning: `torc

Episode: 0, Reward: -21.00, Epsilon: 0.10
Episode: 0, Reward: -21.00, Epsilon: 0.10
Episode: 0, Reward: -15.00, Epsilon: 0.10
Episode: 0, Reward: -19.00, Epsilon: 0.10
Episode: 0, Reward: -21.00, Epsilon: 0.10
Episode: 0, Reward: -21.00, Epsilon: 0.10
Episode: 0, Reward: -21.00, Epsilon: 0.10
Episode: 0, Reward: -21.00, Epsilon: 0.10


  1%|▏         | 2/150 [04:27<5:31:07, 134.24s/it]

Episode: 1, Reward: -19.00, Epsilon: 0.10
Episode: 1, Reward: -19.00, Epsilon: 0.10
Episode: 1, Reward: -20.00, Epsilon: 0.10
Episode: 1, Reward: -20.00, Epsilon: 0.10
Episode: 1, Reward: -21.00, Epsilon: 0.10
Episode: 1, Reward: -19.00, Epsilon: 0.10
Episode: 1, Reward: -19.00, Epsilon: 0.10
Episode: 1, Reward: -19.00, Epsilon: 0.10


  2%|▏         | 3/150 [06:36<5:22:41, 131.71s/it]

...

Episode: 147, Reward: -19.00, Epsilon: 0.10
Episode: 147, Reward: -15.00, Epsilon: 0.10
Episode: 147, Reward: -19.00, Epsilon: 0.10
Episode: 147, Reward: -19.00, Epsilon: 0.10
Episode: 147, Reward: -19.00, Epsilon: 0.10
Episode: 147, Reward: -19.00, Epsilon: 0.10
Episode: 147, Reward: -21.00, Epsilon: 0.10
Episode: 147, Reward: -21.00, Epsilon: 0.10


 99%|█████████▉| 149/150 [4:38:46<01:49, 109.14s/it]

Episode: 148, Reward: -21.00, Epsilon: 0.10
Episode: 148, Reward: -21.00, Epsilon: 0.10
Episode: 148, Reward: -21.00, Epsilon: 0.10
Episode: 148, Reward: -21.00, Epsilon: 0.10
Episode: 148, Reward: -21.00, Epsilon: 0.10
Episode: 148, Reward: -21.00, Epsilon: 0.10
Episode: 148, Reward: -21.00, Epsilon: 0.10
Episode: 148, Reward: -21.00, Epsilon: 0.10


100%|██████████| 150/150 [4:40:35<00:00, 112.24s/it]

Episode: 149, Reward: -21.00, Epsilon: 0.10
Episode: 149, Reward: -21.00, Epsilon: 0.10
Episode: 149, Reward: -21.00, Epsilon: 0.10
Episode: 149, Reward: -21.00, Epsilon: 0.10
Episode: 149, Reward: -21.00, Epsilon: 0.10
Episode: 149, Reward: -17.00, Epsilon: 0.10
Episode: 149, Reward: -21.00, Epsilon: 0.10
Episode: 149, Reward: -16.00, Epsilon: 0.10


In [ ]:
# Create a new instance of your DQNAgent
# agent = DQNAgent(envs)  # Initialize with the same environment and parameters

# # Load the state dictionaries into your model
# agent.policy_net.load_state_dict(torch.load('dqn_policy_net.pth'))
# agent.target_net.load_state_dict(torch.load('dqn_target_net.pth'))
# agent.optimizer.load_state_dict(torch.load('dqn_optimizer.pth'))

# # Load the history list
# history = np.load('dqn_history.npy', allow_pickle=True)  # Load the history list
